In [ ]:
!pip3 install flask


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\N0trUe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


: 

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit


In [1]:
!pip3 install joblib
!pip3 install pandas
!pip3 install scikit-learn



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\N0trUe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\N0trUe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\N0trUe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip3 freeze > requirements.txt
!pip3 install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\N0trUe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip3 install tqdm


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\N0trUe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
!pip3 --version

pip 24.0 from C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip (python 3.11)



In [ ]:
from flask import Flask, request, jsonify
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from scipy.sparse import hstack
import numpy as np
import logging

# Initialize Flask app
app = Flask(__name__)

# Set up logging to a file
logging.basicConfig(filename='app.log', level=logging.ERROR, format='%(asctime)s %(levelname)s: %(message)s')

# Load the trained model and vectorizer
try:
    print("Loading model...")
    model = joblib.load('fine_tuned_ensemble_model2.pkl')
    print("Model loaded.")

    print("Loading vectorizer...")
    vectorizer = joblib.load('vectorizer2.pkl')
    print("Vectorizer loaded.")
except Exception as e:
    app.logger.error(f"Error loading model or vectorizer: {e}")
    raise e

# Initialize a dictionary to track counts of logs by IP and URL
log_counts = defaultdict(lambda: defaultdict(int))  # {ip: {url: count}}

# Helper function to parse individual log entry
def parse_log_entry(log):
    try:
        # Try splitting by `|` first
        if '|' in log:
            parts = log.split("|")
        else:
            # Fall back to space splitting if `|` is not found
            parts = log.split(" ")

        if len(parts) < 6:
            raise ValueError("Log entry does not contain enough parts")

        ip = parts[0]
        method = parts[1]
        url = parts[2]
        status = parts[3]
        size = parts[4]
        referer = parts[5] if parts[5] != '-' else '-'

        structured_log = f"{ip} {method} {url} {status} {size} {referer}"
        return ip, url, structured_log

    except (IndexError, ValueError) as e:
        print(f"Skipping malformed log entry: {log} - Error: {e}")
        return None, None, None


# Route to predict a single log entry (POST method)
@app.route('/predict', methods=['POST'])
def predict_log():
    try:
        data = request.json
        log = data.get('log')

        if not log:
            return jsonify({'error': 'No log entry provided'}), 400

        # Parse the log and handle malformed cases
        ip, url, parsed_log = parse_log_entry(log)
        if not parsed_log:
            return jsonify({'error': 'Invalid log format'}), 400

        # Get the current count for this IP-URL pair
        count = log_counts[ip][url]  # Use current count without incrementing

        # Transform the log and combine with the count feature
        X_text = vectorizer.transform([parsed_log])
        X_count = np.array([[count]])
        X_test = hstack((X_text, X_count))

        # Predict the label using the model
        prediction = model.predict(X_test)[0]
        label_mapping = {0: 'normal', 1: 'suspicious status', 2: 'bypass', 3: 'brute force'}
        predicted_label = label_mapping.get(prediction, 'unknown')

        # Increment the count after prediction
        log_counts[ip][url] += 1

        # Return the result
        result = {
            'log': log,
            'count': count + 1,  # Report the new count after increment
            'predicted_label': predicted_label
        }
        return jsonify(result)

    except Exception as e:
        app.logger.error(f"Error in /predict route: {e}")
        return jsonify({'error': 'Internal Server Error'}), 500

# Route to reset the log counts
@app.route('/reset', methods=['POST'])
def reset_counts():
    global log_counts
    log_counts = defaultdict(lambda: defaultdict(int))
    return jsonify({'message': 'Log counts reset successfully'})

# Route to test if API is running
@app.route('/', methods=['GET'])
def home():
    try:
        return "Threat Classification API is running!"
    except Exception as e:
        app.logger.error(f"Error in / route: {e}")
        return jsonify({'error': 'Internal Server Error'}), 500

# New GET method to predict log entry through query parameters
@app.route('/predict_get', methods=['GET'])
def predict_log_get():
    try:
        log = request.args.get('log')

        if not log:
            return jsonify({'error': 'No log entry provided'}), 400

        # Parse the log and handle malformed cases
        ip, url, parsed_log = parse_log_entry(log)
        if not parsed_log:
            return jsonify({'error': 'Invalid log format'}), 400

        # Get the current count for this IP-URL pair
        count = log_counts[ip][url]

        # Transform the log and combine with the count feature
        X_text = vectorizer.transform([parsed_log])
        X_count = np.array([[count]])
        X_test = hstack((X_text, X_count))

        # Predict the label using the model
        prediction = model.predict(X_test)[0]
        label_mapping = {0: 'normal', 1: 'suspicious status', 2: 'bypass', 3: 'brute force'}
        predicted_label = label_mapping.get(prediction, 'unknown')

        # Increment the count after prediction
        log_counts[ip][url] += 1

        # Return the result
        result = {
            'log': log,
            'count': count + 1,
            'predicted_label': predicted_label
        }
        return jsonify(result)

    except Exception as e:
        app.logger.error(f"Error in /predict_get route: {e}")
        return jsonify({'error': 'Internal Server Error'}), 500

# Run the Flask app
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8080)

Loading model...
Model loaded.
Loading vectorizer...
Vectorizer loaded.
 * Serving Flask app '__main__'
 * Debug mode: off
